<a href="https://colab.research.google.com/github/asongulol/big_data/blob/master/Take__2_ETL_Challenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-2.4.6'
spark_version = 'spark-2.4.6'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:4 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:5 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease [15.4 kB]
Get:6 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:7 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,626 B]
Ign:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:12 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic/main Sources [1,864

In [2]:
# Download Postgresql driver
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2020-09-01 19:05:55--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.1’

postgresql-42.2.9.j 100%[===================>] 892.61K  1.43MB/s    in 0.6s    

2020-09-01 19:05:56 (1.43 MB/s) - ‘postgresql-42.2.9.jar.1’ saved [914037/914037]



In [3]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [4]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer, StopWordsRemover

# Extract Transform and Load (ETL)

## Extract

In [5]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url ="https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Electronics_v1_00.tsv.gz"
spark.sparkContext.addFile(url)

# Create a dataframe
df = spark.read.option('header', 'true').csv(SparkFiles.get("amazon_reviews_us_Electronics_v1_00.tsv.gz"), inferSchema=True, sep="\t", timestampFormat="yyyy-mm-dd", header=True)

# Show DataFrame
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   41409413|R2MTG1GCZLR2DK|B00428R89M|     112201306|yoomall 5M Antenn...|     Electronics|          5|            0|          0|   N|                Y|          Five Stars|       As described.|2015-01-31 00:08:00|
|         US|   49668221|R2HBOEM8LE9928|B000068O48|     734576678|Hosa GPM-103 3.5m...| 

## Transform

In [6]:
# Count number of rows
df.count()

3093869

In [7]:
# Identify if there are enough Vine program reviews
print(df[df['vine'] == 'Y'].count())

18512


In [8]:
# Drop null values
df = df.dropna()

print(df.count())

3093660


In [9]:
# Print schema
df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: timestamp (nullable = true)



In [10]:
# Change datatypes

from pyspark.sql.types import TimestampType, IntegerType

df = df.withColumn("customer_id", df["customer_id"].cast(IntegerType()))\
       .withColumn("product_parent", df["product_parent"].cast(IntegerType()))\
       .withColumn("review_date", df["review_date"].cast(TimestampType()))\
       .withColumn("star_rating", df["star_rating"].cast(IntegerType()))\
       .withColumn("helpful_votes", df["helpful_votes"].cast(IntegerType()))\
      .withColumn("total_votes", df["total_votes"].cast(IntegerType()))\

# Print schema
df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: timestamp (nullable = true)



In [11]:
# Create review dataframe to match review_id_table table
review_id_df = df.select(["review_id","customer_id","product_id", "product_parent","review_date"])
review_id_df.show()

+--------------+-----------+----------+--------------+-------------------+
|     review_id|customer_id|product_id|product_parent|        review_date|
+--------------+-----------+----------+--------------+-------------------+
|R2MTG1GCZLR2DK|   41409413|B00428R89M|     112201306|2015-01-31 00:08:00|
|R2HBOEM8LE9928|   49668221|B000068O48|     734576678|2015-01-31 00:08:00|
|R1P4RW1R9FDPEE|   12338275|B000GGKOG8|     614448099|2015-01-31 00:08:00|
|R1EBPM82ENI67M|   38487968|B000NU4OTA|      72265257|2015-01-31 00:08:00|
|R372S58V6D11AT|   23732619|B00JOQIO6S|     308169188|2015-01-31 00:08:00|
|R1A4514XOYI1PD|   21257820|B008NCD2LG|     976385982|2015-01-31 00:08:00|
|R20D9EHB7N20V6|    3084991|B00007FGUF|     670878953|2015-01-31 00:08:00|
|R1WUTD8MVSROJU|    8153674|B00M9V2RMM|     508452933|2015-01-31 00:08:00|
|R1QCYLT25812DM|   52246189|B00J3O9DYI|     766372886|2015-01-31 00:08:00|
| R904DQPBCEM7A|   41463864|B00NS1A0E4|     458130381|2015-01-31 00:08:00|
|R1DGA6UQIVLKZ7|    27819

In [12]:
# Print schema
review_id_df.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- review_date: timestamp (nullable = true)



In [13]:
# Create product dataframe to match products table
products_df = df.select(["product_id","product_title"])
products_df = products_df.dropDuplicates()
products_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00JL2498C|Jensen JTA-410-BL...|
|B00OUKG3NE|Sennheiser Androi...|
|B0083H9R6A|Electrified ELPLP...|
|B0097BEE9Q|Apple iPod nano 1...|
|B0037Z4MCM|Pyle Marine Sport...|
|B00P0BQKBC|Water & Wood Lock...|
|B00TP1C1UC|AmazonBasics 6-Ou...|
|B005AIF0X4|iHip MVF1030WO Ma...|
|B009JJKZSK|Click 'N Dig! Key...|
|B00KXDS4VQ|enKo products HDM...|
|B009W857SG|TETC Cable (3 Fee...|
|B007LG453Y|Tmvel TVM-UK2 Uni...|
|B00KFSO0SA|Monoprice 11149 3...|
|B00IH5U050|Digital Army Camo...|
|B00JDYMPYS|Sylvania SKCR2713...|
|B00YDMW0DQ|Sound Intone HD30...|
|B00AR0VCXW|Pro-Ject Speed Bo...|
|B006BE6WK6|12 PIECE Ear Hook...|
|B00TM1CT98|Bose Acoustimass ...|
|B00P8D911I|AuraBeam Samsung ...|
+----------+--------------------+
only showing top 20 rows



In [14]:
# Create customer dataframe to match customers table
from pyspark.sql.functions import desc

customers_df = df.groupby("customer_id").agg({"customer_id":"count"})
customers_df = customers_df.orderBy(desc("count(customer_id)"))
customers_df = customers_df.withColumnRenamed("count(customer_id)", "customer_count") 
customers_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   49266466|           234|
|   53075795|           200|
|   30669680|           190|
|   53037408|           180|
|   50820654|           171|
|   52938899|           166|
|   50027179|           159|
|   44834233|           159|
|   39789300|           154|
|   32038204|           149|
|   53090839|           140|
|   52340667|           135|
|   51346302|           132|
|   50530418|           132|
|   52870270|           118|
|   12200139|           117|
|   52643790|           113|
|   52460215|           112|
|   51812418|           112|
|   50529300|           107|
+-----------+--------------+
only showing top 20 rows



In [15]:
# Create vine dataframe to match vine_table table

# Load in a sql function to use columns
from pyspark.sql.functions import col

vine_table_df = df.select(["review_id","star_rating","helpful_votes", "total_votes","vine"])

# Filter for only columns from Amazon's Vine program
#vine_table_df = vine_table_df.filter(col("vine")  == "Y")
vine_table_df.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R2MTG1GCZLR2DK|          5|            0|          0|   N|
|R2HBOEM8LE9928|          5|            0|          0|   N|
|R1P4RW1R9FDPEE|          5|            1|          1|   N|
|R1EBPM82ENI67M|          1|            0|          0|   N|
|R372S58V6D11AT|          5|            1|          1|   N|
|R1A4514XOYI1PD|          5|            1|          1|   N|
|R20D9EHB7N20V6|          5|            0|          0|   N|
|R1WUTD8MVSROJU|          5|            0|          0|   N|
|R1QCYLT25812DM|          4|            0|          0|   N|
| R904DQPBCEM7A|          4|            0|          0|   N|
|R1DGA6UQIVLKZ7|          5|            0|          0|   N|
| RLQT3V8SMNIBH|          5|            0|          0|   N|
|R3T9GZS2TMXZGM|          1|            0|          0|   N|
|R24HVAEYP5PLDN|          5|            

In [16]:
# Filter for only columns from Amazon's Vine program
Y_vine_table_df = vine_table_df.filter(col("vine")  == "Y")
Y_vine_table_df.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R1T7X1RPWYTRBU|          5|            2|          4|   Y|
| RJFGZBIFXCC65|          5|            0|          0|   Y|
| RSZ6RLTTGWUN6|          5|            0|          1|   Y|
|R2N4GVP3Q5PF9Y|          3|            2|          3|   Y|
| RCAFL1X52MFZM|          5|            1|          2|   Y|
|R17D8E4KO0FS83|          5|            1|          1|   Y|
|R184FOUNZZ7KO8|          5|           15|         20|   Y|
|R2SP52AJMIERZ5|          3|            0|          0|   Y|
|R15VB3EZ2C0W3J|          4|            0|          0|   Y|
|R3CMY924EQEGIU|          4|            0|          0|   Y|
|R2T1AAO75DGP6C|          4|            0|          0|   Y|
|R2AYZV2RYCKIDO|          4|            1|          1|   Y|
|R3BEI48BY6XIRV|          3|            0|          0|   Y|
|R2PLGJM0MUI41A|          5|            

## Load

In [17]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://dataviz.clw759vfh14c.us-east-2.rds.amazonaws.com:5432/my_data_class_db"
config = {"user":"postgres", 
          "password": "f9dLM910&%S%", 
          "driver":"org.postgresql.Driver"}

In [18]:
# Write DataFrame to review_id_df table in RDS

#review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [19]:
# Write DataFrame to products table in RDS

#products_df.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [20]:
# Write DataFrame to customers table in RDS

#customers_df.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [21]:

# Write DataFrame to vine_table table in RDS

#vine_table_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)

## Analysis of Whether Vine Reviews Are Trustworthy

In [4]:
# Load dependencies
!pip install psycopg2-binary
import psycopg2


     |████████████████████████████████| 2.9MB 2.8MB/s 


In [38]:
# Connect to Postgres db
try:
    connection = psycopg2.connect(user = "postgres",
                                  password = "f9dLM910&%S%",
                                  host = "dataviz.clw759vfh14c.us-east-2.rds.amazonaws.com",
                                  port = "5432",
                                  database = "my_data_class_db")

    cursor = connection.cursor()
    # Print PostgreSQL Connection properties
    print ( connection.get_dsn_parameters(),"\n")

    # Print PostgreSQL version
    cursor.execute("SELECT version();")
    record = cursor.fetchone()
    print("You are connected to - ", record,"\n")

except (Exception, psycopg2.Error) as error :
    print ("Error while connecting to PostgreSQL", error)

{'user': 'postgres', 'dbname': 'my_data_class_db', 'host': 'dataviz.clw759vfh14c.us-east-2.rds.amazonaws.com', 'port': '5432', 'tty': '', 'options': '', 'sslmode': 'prefer', 'sslcompression': '0', 'krbsrvname': 'postgres', 'target_session_attrs': 'any'} 

You are connected to -  ('PostgreSQL 11.6 on x86_64-pc-linux-gnu, compiled by gcc (GCC) 4.8.3 20140911 (Red Hat 4.8.3-9), 64-bit',) 



In [67]:
   
   postgreSQL_select_Query = "select * from vine_table where vine = 'Y'"

   cursor.execute(postgreSQL_select_Query)
   print("Selecting rows from vine using cursor.fetchall")
   vine = cursor.fetchall() 


Selecting rows from vine using cursor.fetchall


In [45]:
import pandas as pd

vine_df = pd.DataFrame(vine, columns=["review_id", "star_rating", "helpful_votes", "total_votes", "vine"])

In [68]:
vine_df

,review_id,star_rating,helpful_votes,total_votes,vine
0,R1T7X1RPWYTRBU,5,2,4,Y
1,RJFGZBIFXCC65,5,0,0,Y
2,RSZ6RLTTGWUN6,5,0,1,Y
3,R2N4GVP3Q5PF9Y,3,2,3,Y
4,RCAFL1X52MFZM,5,1,2,Y
...,...,...,...,...,...
18507,RD4IPYAK8Z7NJ,4,6,9,Y
18508,R3BJ8VBZ0FVAH9,4,21,24,Y
18509,R3PZCB4HLG1VM3,5,19,21,Y
18510,R2KKSA5G8A8V9B,4,25,30,Y


In [94]:
print(vine_df.describe())

        star_rating  helpful_votes   total_votes
count  18512.000000   18512.000000  18512.000000
mean       4.135048       5.824654      7.212565
std        0.962940      37.449657     40.058811
min        1.000000       0.000000      0.000000
25%        4.000000       0.000000      0.000000
50%        4.000000       1.000000      2.000000
75%        5.000000       3.000000      5.000000
max        5.000000    2561.000000   2688.000000


In [56]:
   postgreSQL_select_Query = "select * from vine_table where vine = 'N'"

   cursor.execute(postgreSQL_select_Query)
   print("Selecting rows from not vine using cursor.fetchall")
   not_vine = cursor.fetchall() 
 

Selecting rows from not vine using cursor.fetchall


In [64]:
not_vine_df = pd.DataFrame(not_vine, columns=["review_id", "star_rating", "helpful_votes", "total_votes", "vine"])
print(not_vine_df)

              review_id  star_rating  helpful_votes  total_votes vine
0        R2MTG1GCZLR2DK            5              0            0    N
1        R2HBOEM8LE9928            5              0            0    N
2        R1P4RW1R9FDPEE            5              1            1    N
3        R1EBPM82ENI67M            1              0            0    N
4        R372S58V6D11AT            5              1            1    N
...                 ...          ...            ...          ...  ...
3075143   RZ3LLAKIZQ3RR            5              0            0    N
3075144  R1V8GVI67B7V0K            5              0            0    N
3075145  R3BDESK5VUNK8C            5              5            5    N
3075146  R1JJGM0JWVT7DV            5              3            3    N
3075147  R30PZG0PCGR2V0            1              7            9    N

[3075148 rows x 5 columns]


In [95]:
print(not_vine_df.describe())

        star_rating  helpful_votes   total_votes
count  3.075148e+06   3.075148e+06  3.075148e+06
mean   4.034911e+00   1.835739e+00  2.341779e+00
std    1.389569e+00   2.119184e+01  2.233693e+01
min    1.000000e+00   0.000000e+00  0.000000e+00
25%    3.000000e+00   0.000000e+00  0.000000e+00
50%    5.000000e+00   0.000000e+00  0.000000e+00
75%    5.000000e+00   1.000000e+00  1.000000e+00
max    5.000000e+00   1.278600e+04  1.294400e+04


In [73]:
# Do a t-test of the means between Vine and Non-vine Reviews
## Load dependencies
import scipy.stats
from scipy.stats import ttest_ind

In [90]:
# Compare the star rating between Vine and Non-Vine groups using t-test
vine_star_rating = vine_df['star_rating']
not_vine_star_rating = not_vine_df['star_rating']
t, p = ttest_ind(vine_star_rating, not_vine_star_rating)
print("t = " + str(t))
print("p = " + str(p))

t = 9.79064893112825
p = 1.2359432482650555e-22


In [91]:
# Compare the helpful votes using t-test
vine_helpful_votes = vine_df['helpful_votes']
not_vine_helpful_votes = not_vine_df['helpful_votes']
t1, p1 = ttest_ind(vine_helpful_votes, not_vine_helpful_votes)
print("t = " + str(t1))
print("p = " + str(p1))

t = 25.372810330639105
p = 5.206488915984731e-142


In [92]:
# Compare the total votes using t-test
vine_total_votes = vine_df['total_votes']
not_vine_total_votes = not_vine_df['total_votes']
t2, p2 = ttest_ind(vine_total_votes, not_vine_total_votes)
print("t = " + str(t2))
print("p = " + str(p2))

t = 29.385853464583047
p = 8.845451954852947e-190


In [93]:
# Close Postgres connection
try:
    connection = psycopg2.connect(user = "postgres",
                                  password = "f9dLM910&%S%",
                                  host = "dataviz.clw759vfh14c.us-east-2.rds.amazonaws.com",
                                  port = "5432",
                                  database = "my_data_class_db")

finally:
    #closing database connection.
        if(connection):
            cursor.close()
            connection.close()
            print("PostgreSQL connection is closed")

PostgreSQL connection is closed
